In [1]:
import json
import os
import re
import logging
import time
import requests


def get_connections(domain, url):

    conn_dict = dict()

    dir_path = os.path.dirname(os.path.abspath(__file__))

    with open(f'{dir_path}/../.secrets/looker_secrets.json') as f:
        j = json.load(f)

    data = {
        'client_id': j['client_id'],
        'client_secret': j['client_secret']
    }

    response = requests.post(f'{domain}/login', data=data)

    access_token = response.json()['access_token']

    headers = {
        'Authorization': f'token {access_token}'
    }

    response = requests.get(domain+url, headers=headers)

    for r in response.json():
        conn_dict[r['name']] = {'database': r['database'], 'type': r['dialect']['label']}

    return conn_dict


def look_up_target_view(source_view_name, view_map):
    target_view_payload = view_map[source_view_name]
    return target_view_payload


def is_true_source(source):
    return bool(len(source.split('.')) == 3)


def get_conn_db(explore, connection_map):
    conn = explore['conn']
    database = connection_map[conn]['database']
    provider = connection_map[conn]['type'].split(' ')
    if len(provider) > 1:
        provider = provider[1]
    else:
        provider = provider[0]
        
    return provider, database

####### need to add NDT, and derived table ################
def get_true_source(dir_path, view_payload, explore, connection_map, view_map):
    
    if isinstance(view_payload, dict):

        if view_payload['view_type'] == 'extension': 

            new_view_name = look_up_target_view(view_payload['source_table'], view_map=view_map)
            with open(f'{dir_path}/../maps/{new_view_name}', 'r') as f:
                new_view_payload = json.load(f)
            return get_true_source(dir_path, new_view_payload, explore, connection_map, view_map)
            
        elif view_payload['view_type'] == 'sql_table_name': 

            source = view_payload['source_table']

            if is_true_source(source):
                true_source = view_payload['source_table']

            else: 
                if len(source.split('.')) == 3:
                    true_source = source
                
                else:
                    provider, database = get_conn_db(explore=explore, connection_map=connection_map)
                    if len(source.split('.')) == 2:
                        true_source = f'{provider}.{source}'
                    elif len(source.split('.')) == 1:
                        true_source = f'{provider}.{database}.{source}'

        elif view_payload['view_type'] == 'derived_table':
            true_source = 'derived table'

    else:
        true_source = view_payload

    return view_payload['view_name'], true_source


In [2]:
view_content = dict()
view_map = dict()
for view in next(os.walk(f'../maps'))[2]:
    if view.startswith('view'):
 
        view_map[view.split('-')[1].split('.')[0]] = view
        with open(f'../maps/{view}','r') as f:
            payload = json.load(f)
            view_content[payload['view_name']] = payload


In [3]:
view_map

{'sf__leads': 'view-sf__leads.json',
 'sf__cases': 'view-sf__cases.json',
 'sfbase__users': 'view-sfbase__users.json',
 'total_active_node_count': 'view-total_active_node_count.json',
 'sf__accounts': 'view-sf__accounts.json',
 'snowflake_sf__accounts': 'view-snowflake_sf__accounts.json',
 'sfbase__cases': 'view-sfbase__cases.json',
 'sf__contacts': 'view-sf__contacts.json',
 'sfbase__leads': 'view-sfbase__leads.json',
 'sfbase__contacts': 'view-sfbase__contacts.json',
 'sfbase__accounts': 'view-sfbase__accounts.json',
 'sf__users': 'view-sf__users.json'}

In [5]:
get_connections(domain='https://docker.looker.com:19999', url='/api/3.1/connections')

NameError: name '__file__' is not defined

In [7]:
domain='https://docker.looker.com:19999'
url='/api/3.1/connections'

In [9]:
conn_dict = dict()

In [10]:
with open(f'../../.secrets/looker_secrets.json') as f:
    j = json.load(f)

data = {
    'client_id': j['client_id'],
    'client_secret': j['client_secret']
}

response = requests.post(f'{domain}/login', data=data)

access_token = response.json()['access_token']

headers = {
    'Authorization': f'token {access_token}'
}

response = requests.get(domain+url, headers=headers)

for r in response.json():
    conn_dict[r['name']] = {'database': r['database'], 'type': r['dialect']['label']}

In [11]:
conn_dict

{'hub': {'database': 'accounts', 'type': 'PostgreSQL'},
 'registry-hub': {'database': 'repos', 'type': 'PostgreSQL'},
 'data_warehouse': {'database': 'production', 'type': 'Amazon Redshift'},
 'docker_cloud': {'database': 'tutum', 'type': 'MySQL'},
 'nautilus_db': {'database': 'nautilus', 'type': 'PostgreSQL'},
 'restricted': {'database': 'restricted', 'type': 'Amazon Redshift'},
 'data_warehouse_staging': {'database': 'staging', 'type': 'Amazon Redshift'},
 'snowflake_segment': {'database': 'SEGMENT', 'type': 'Snowflake'},
 'snowflake_production': {'database': 'PRODUCTION', 'type': 'Snowflake'},
 'bigquery': {'database': 'clari_pulse_breakdown',
  'type': 'Google BigQuery Standard SQL'},
 'bigquery_production': {'database': 'finance',
  'type': 'Google BigQuery Standard SQL'},
 'snowflake_medium': {'database': 'PRODUCTION', 'type': 'Snowflake'},
 'initiatives-db': {'database': 'initiatives', 'type': 'PostgreSQL'},
 'bigquery_production_marketing': {'database': 'marketing',
  'type': '